In [20]:
# OPERATING SYSTEM STUFF
import os
import io
import gc

# DATA SCIENCE
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# API STUFF
import xlrd
import requests
import json

# SQL
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError # ProgrammingError catches SQL write exceptions
from sqlalchemy import String, Integer, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table, select
from sqlalchemy.sql import and_
from sqlalchemy import create_engine, MetaData, Table, select

# GEOCODING
from geopy.geocoders import GoogleV3

# CONFIGURATION FILES
import config
pd.set_option('display.float_format', '{:.6f}'.format)

# OTHER
from tqdm.notebook import tqdm

In [41]:
# FUNCTION DECLARATIONS

def geolocate(row):
    if not row['GEOCODING ERR']:  # If GEOCODING ERR is False, run the geocoding API
        address = ', '.join([row['ADDRESS'],
                             #row['NEIGHBORHOOD'],
                             row['BOROUGH']]) + ', New York City'
        response = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={config.GOOGLE_API_KEY}')
        res = response.json()
        if res['results']:
            location = res['results'][0]
            if location.get('partial_match'):  # Check for partial match
                row['GEOCODING ERR'] = True
                row['LATITUDE'] = None
                row['LONGITUDE'] = None
            else:
                row['LATITUDE'] = location['geometry']['location']['lat']
                row['LONGITUDE'] = location['geometry']['location']['lng']
        else:
            row['GEOCODING ERR'] = True  # Update GEOCODING ERR to True if geolocation failed
            row['LATITUDE'] = None
            row['LONGITUDE'] = None
    return row

def print_sql_table(engine, table_name):
    """
    This function retrieves and prints all the rows from a SQL table.

    Parameters:
    engine (sqlalchemy.engine.Engine): SQLAlchemy engine instance.
    table_name (str): Name of the table in the SQL database to be printed.

    Returns:
    None. The function prints the rows of the SQL table.
    """
    metadata = MetaData()

    # Reflect the table
    table = Table(table_name, metadata, autoload_with=engine)

    # Connect to the engine
    with engine.connect() as connection:
        # Select all rows from the table
        stmt = select([table])
        result = connection.execute(stmt)

        # Print the rows
        for row in result:
            print(row)

# def upload_geocoding_dataframe_to_sql(engine, table_name, df):
#     """
#     This function uploads a pandas DataFrame to a SQL table, either inserting new rows or updating existing ones.
#     The DataFrame must contain the following columns: 'BOROUGH CODE', 'BOROUGH', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'GEOCODING ERR'.
    
#     Rows from the DataFrame are compared to existing rows in the SQL table based on 'BOROUGH CODE', 'BOROUGH', and 'ADDRESS'.
#     If a matching row is found in the SQL table, it is updated with the data from the DataFrame. If no matching row is found, a new row is inserted.

#     Parameters:
#     engine (sqlalchemy.engine.Engine): SQLAlchemy engine instance.
#     table_name (str): Name of the table in the SQL database to which the DataFrame will be uploaded.
#     df (pandas.DataFrame): DataFrame to be uploaded. Must contain the required columns.

#     Returns:
#     None. The function operates in-place on the SQL table.
#     """
#     metadata = MetaData()

#     # Reflect the table
#     table = Table(table_name, metadata, autoload_with=engine)

#     # Replace NaN values with None
#     df = df.where(pd.notnull(df), None)

#     # Connect to the engine
#     with engine.connect() as connection:
#         for i, row in df.iterrows():
#             # Check if the row exists
#             stmt = select([table]).where(
#                 and_(
#                     table.c['BOROUGH CODE'] == row['BOROUGH CODE'],
#                     table.c['BOROUGH'] == row['BOROUGH'],
#                     table.c['ADDRESS'] == row['ADDRESS']
#                 )
#             )
#             result = connection.execute(stmt).fetchone()
            
#             # If the row exists, update it
#             if result:
#                 update_values = {
#                     'BOROUGH CODE': row['BOROUGH CODE'],
#                     'BOROUGH': row['BOROUGH'],
#                     'ADDRESS': row['ADDRESS'],
#                     'LATITUDE': row['LATITUDE'],
#                     'LONGITUDE': row['LONGITUDE'],
#                     'GEOCODING ERR': row['GEOCODING ERR']
#                 }
#                 stmt = (
#                     table.update().
#                     where(
#                         and_(
#                             table.c['BOROUGH CODE'] == row['BOROUGH CODE'],
#                             table.c['BOROUGH'] == row['BOROUGH'],
#                             table.c['ADDRESS'] == row['ADDRESS']
#                         )
#                     ).
#                     values(**update_values)
#                 )
#             # If the row doesn't exist, insert it
#             else:
#                 stmt = table.insert().values(**row.to_dict())

#             # Execute the statement
#             connection.execute(stmt)

def upload_geocoding_dataframe_to_sql(engine, table_name, df):
    """
    This function uploads a pandas DataFrame to a SQL table, either inserting new rows or updating existing ones.
    The DataFrame must contain the following columns: 'BOROUGH CODE', 'BOROUGH', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'GEOCODING ERR'.
    
    Rows from the DataFrame are compared to existing rows in the SQL table based on 'BOROUGH CODE', 'BOROUGH', and 'ADDRESS'.
    If a matching row is found in the SQL table, it is updated with the data from the DataFrame. If no matching row is found, a new row is inserted.

    Parameters:
    engine (sqlalchemy.engine.Engine): SQLAlchemy engine instance.
    table_name (str): Name of the table in the SQL database to which the DataFrame will be uploaded.
    df (pandas.DataFrame): DataFrame to be uploaded. Must contain the required columns.

    Returns:
    None. The function operates in-place on the SQL table.
    """
    df = df.where(pd.notnull(df), None)

    # Create a primary key in the dataframe
    df['PRIMARY_KEY'] = df['BOROUGH CODE'].astype(str) + '_' + df['BOROUGH'] + '_' + df['ADDRESS']

    # Try to insert new rows
    try:
        df.to_sql(table_name, con=engine, index=False, if_exists='append')
    except Exception as e:
        print("Insert operation failed. This may be due to duplicate keys. Now attempting update operation...")
    
    # Now let's handle the update operation. 
    with engine.connect() as connection:
        for i, row in df.iterrows():
            stmt = text(
                f"""
                UPDATE {table_name}
                SET
                [BOROUGH CODE] = :borough_code,
                [BOROUGH] = :borough,
                [ADDRESS] = :address,
                [LATITUDE] = :latitude,
                [LONGITUDE] = :longitude,
                [GEOCODING ERR] = :geocoding_err
                WHERE PRIMARY_KEY = :primary_key
                """
            )
            connection.execute(stmt, **row.to_dict())


In [22]:
os.environ['SQLALCHEMY_WARN_20'] = '0'
os.environ['SQLALCHEMY_SILENCE_UBER_WARNING'] = '1'

# Database params & credentials
username = 'root'
password = 'rootpassword'
hostname = 'db'
database_name = 'database_1'

# Create database connection
engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}')

try:
    with engine.connect() as connection:
        connection.execute(text(f'CREATE DATABASE {database_name};'))
except ProgrammingError:
    pass

engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}')

# Show databases
# with engine.connect() as connection:
#     result = connection.execute(text("SHOW DATABASES;"))
#     databases = [row[0] for row in result]
#     print(databases)

In [23]:
# Generate mapping between NYC data and Zillow categories.

# Show all the building classes
# This won't work unless the DataFrame 'combined' has been set up
# sorted_building_classes = sorted(
#     combined["BUILDING CLASS CATEGORY"].unique(),
#     key=lambda x: int(x.split(" ")[0])
# )

# Define the mappings
mapping = {
    "Single-family home": ['01 ONE FAMILY DWELLINGS'],
    "Multi-family home": [
        '03 THREE FAMILY DWELLINGS',
        '07 RENTALS - WALKUP APARTMENTS',
        '08 RENTALS - ELEVATOR APARTMENTS',
        '14 RENTALS - 4-10 UNIT'
    ],
    "Apartment": [
        '07 RENTALS - WALKUP APARTMENTS',
        '08 RENTALS - ELEVATOR APARTMENTS',
        '09 COOPS - WALKUP APARTMENTS',
        '10 COOPS - ELEVATOR APARTMENTS'
    ],
    "Condo": [
        '04 TAX CLASS 1 CONDOS',
        '12 CONDOS - WALKUP APARTMENTS',
        '13 CONDOS - ELEVATOR APARTMENTS',
        '15 CONDOS - 2-10 UNIT RESIDENTIAL',
        '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT'
    ],
    "Co-op": [
        '09 COOPS - WALKUP APARTMENTS',
        '10 COOPS - ELEVATOR APARTMENTS',
        '17 CONDO COOPS'
    ],
    "Duplex": ['02 TWO FAMILY DWELLINGS'],
    "Townhouse": [
        '01 ONE FAMILY DWELLINGS',
        '02 TWO FAMILY DWELLINGS'
    ],
    "Brownstone": [
        '01 ONE FAMILY DWELLINGS',
        '02 TWO FAMILY DWELLINGS'
    ],
    "Row house": [
        '01 ONE FAMILY DWELLINGS',
        '02 TWO FAMILY DWELLINGS'
    ],
}

# Flatten the mapping dictionary to create a dataframe
mapping_list = [(k, v) for k, vals in mapping.items() for v in vals]
mapping_df = pd.DataFrame(
    mapping_list,
    columns=['ZILLOW CATEGORY', 'BUILDING CLASS CATEGORY']
)

In [24]:
mapping_df.to_sql('cat_map', con=engine, index=False, if_exists='replace')

#Show tables
# with engine.connect() as connection:
#     result = connection.execute(text("SHOW TABLES;"))
#     tables = [row[0] for row in result]
#     print(tables)

#Show columns from table
with engine.connect() as connection:
    result = connection.execute(text("SHOW COLUMNS FROM geocodes;"))
    tables = [(row[0],row[1]) for row in result]
    print(tables)

[('BOROUGH CODE', 'int(11)'), ('BOROUGH', 'varchar(25)'), ('NEIGHBORHOOD', 'varchar(100)'), ('ADDRESS', 'varchar(255)'), ('LATITUDE', 'float'), ('LONGITUDE', 'float'), ('GEOCODING ERR', 'tinyint(1)')]


In [25]:
# URL Schema
# [Manhattan, Bronx, Brooklyn, Queens, Staten Island]

dataURLs = [
    'https://www.nyc.gov/assets/finance/downloads/pdf/rolling_sales/'
    'rollingsales_manhattan.xlsx',
    'https://www.nyc.gov/assets/finance/downloads/pdf/rolling_sales/'
    'rollingsales_bronx.xlsx',
    'https://www.nyc.gov/assets/finance/downloads/pdf/rolling_sales/'
    'rollingsales_brooklyn.xlsx',
    'https://www.nyc.gov/assets/finance/downloads/pdf/rolling_sales/'
    'rollingsales_queens.xlsx',
    'https://www.nyc.gov/assets/finance/downloads/pdf/rolling_sales/'
    'rollingsales_statenisland.xlsx'
]
# Create an empty array that will hold our NYC Housing DataFrames
data = []

# Pull data from the NYC website
for url in dataURLs:
    # Read Excel file and skip the first 4 rows
    df = pd.read_excel(url, skiprows=4, engine="openpyxl")
    data.append(df)

In [26]:
# Combine the dataframes from the nyc housing website
combined = pd.concat(data, ignore_index=True)

# Add borough names

# Rename the 'BOROUGH' column to 'BOROUGH CODE'
combined = combined.rename(columns={'BOROUGH': 'BOROUGH CODE'})
# Define the mapping for borough codes to borough names
borough_mapping = {1: 'MANHATTAN', 2: 'BRONX', 3: 'BROOKLYN', 4: 'QUEENS', 5: 'STATEN ISLAND'}
# Create a new 'BOROUGH' column based on 'BOROUGH CODE'
borough = combined['BOROUGH CODE'].map(borough_mapping)
# Insert the new 'BOROUGH' column into the DataFrame right after the 'BOROUGH CODE' column
combined.insert(loc=1, column='BOROUGH', value=borough)

# Remove bad rows
combined = combined[~combined['ADDRESS'].str.contains('N/A')]

In [27]:
# Write the contents of `combined` to the `sales` SQL table...
combined.to_sql('sales', con=engine, index=False, if_exists='replace')

In [70]:
# CREATE GEOCODING TABLE & COPY TO SQL TABLE

# Table name in the SQL database
geocodes_sql_table = 'geocodes'

data_types_df = {
    'BOROUGH CODE': int,
    'BOROUGH': str,
    'NEIGHBORHOOD': str,
    'ADDRESS': str,
    'LATITUDE': float,
    'LONGITUDE': float,
    'GEOCODING ERR': bool  # New column
}

data_types_sqlalchemy = {
    'BOROUGH CODE': Integer,
    'BOROUGH': String(25),
    'NEIGHBORHOOD': String(100),
    'ADDRESS': String(255),
    'LATITUDE': Float,
    'LONGITUDE': Float,
    'GEOCODING ERR': Boolean  # New column
}

# Template Pandas DataFrame that we use to build the SQL database
geocodes = pd.DataFrame(columns=data_types_df)

with engine.connect() as connection:
    try:
        # Create the table & confirm
        geocodes.to_sql(geocodes_sql_table, engine, # if_exists='replace',
            index=False, dtype=data_types_sqlalchemy)
        
        # Add a primary key column that corresponds to address fields
        connection.execute(text("ALTER TABLE geocodes ADD PRIMARY_KEY VARCHAR(255)"))
        print(f"Table '{geocodes_sql_table}' created in database '{database_name}'.")
    
    except ValueError:
        print(f"Table '{geocodes_sql_table}' already exists in database '{database_name}'.")
    
    except OperationalError:
        print(f"Table '{geocodes_sql_table}' in '{database_name}' already has a PRIMARY_KEY.")
    
    # Set value of primary key to concatenation of address fields
    connection.execute(
        text(
            "UPDATE geocodes SET PRIMARY_KEY = CONCAT(`BOROUGH CODE`, '_', `BOROUGH`, '_', `ADDRESS`)"
        )
    )
    
    

Table 'geocodes' already exists in database 'database_1'.


In [29]:
# Create a table of geographic information from `combined`
geocodingTable = combined[['BOROUGH CODE', 'BOROUGH', 'NEIGHBORHOOD', 'ADDRESS']].copy()
geocodingTable['LATITUDE'] = None
geocodingTable['LONGITUDE'] = None
geocodingTable['GEOCODING ERR'] = False

In [30]:
# Query the SQL database for the geocodes table
query = f"SELECT * FROM {geocodes_sql_table}"

# Execute the query and load the result into a DataFrame
df_queryResponse = pd.read_sql_query(query, engine)
#df_queryResponse = geocodingTable[:-5].copy() # Fictional, remove in prod

In [31]:
# Find any rows we downloaded from NYC that aren't already in the database
# with a mask
mask = geocodingTable[['BOROUGH CODE', 'BOROUGH', 'NEIGHBORHOOD', 'ADDRESS']].isin(
    df_queryResponse[['BOROUGH CODE', 'BOROUGH', 'NEIGHBORHOOD', 'ADDRESS']]
)
matching_rows = mask.all(axis=1)
missing_rows = geocodingTable[~matching_rows]

# Create a new row with a fictional address
new_row = pd.DataFrame([[5, 'STATEN ISLAND', 'WOODROW', '123 FAKE STREET', None, None, False]],
                       columns=missing_rows.columns)
# Append the new row to the dataframe
missing_rows = missing_rows.append(new_row, ignore_index=True)

In [32]:
df_queryResponse

,BOROUGH CODE,BOROUGH,NEIGHBORHOOD,ADDRESS,LATITUDE,LONGITUDE,GEOCODING ERR
0,5,STATEN ISLAND,WOODROW,40 HERRICK AVENUE,40.530700,-74.219200,0
1,5,STATEN ISLAND,WOODROW,104 GLADWIN STREET,40.531900,-74.222500,0
2,5,STATEN ISLAND,WOODROW,96 LENEVAR AVENUE,40.538900,-74.209400,0
3,5,STATEN ISLAND,WOODROW,401 BLOOMINGDALE ROAD,40.534800,-74.218000,0
4,5,STATEN ISLAND,WOODROW,3120 ARTHUR KILL ROAD,40.543800,-74.233500,0
...,...,...,...,...,...,...,...
100,1,MANHATTAN,ALPHABET CITY,"383 EAST 10TH STREET, B1",40.726500,-73.977800,0
101,1,MANHATTAN,ALPHABET CITY,"383 EAST 10TH STREET, D1",40.726500,-73.977800,0
102,1,MANHATTAN,ALPHABET CITY,"182 EAST 2ND STREET, 1C",40.722700,-73.984400,0
103,1,MANHATTAN,ALPHABET CITY,"503 EAST 13TH STRET, B4",nan,nan,1


In [33]:
missing_rows

,BOROUGH CODE,BOROUGH,NEIGHBORHOOD,ADDRESS,LATITUDE,LONGITUDE,GEOCODING ERR
0,1,MANHATTAN,ALPHABET CITY,347 EAST 4TH STREET,None,None,False
1,1,MANHATTAN,ALPHABET CITY,19 AVENUE D,None,None,False
2,1,MANHATTAN,ALPHABET CITY,110 AVENUE C,None,None,False
3,1,MANHATTAN,ALPHABET CITY,326 EAST 4TH STREET,None,None,False
4,1,MANHATTAN,ALPHABET CITY,328 EAST 4TH STREET,None,None,False
...,...,...,...,...,...,...,...
81399,5,STATEN ISLAND,WOODROW,104 GLADWIN STREET,None,None,False
81400,5,STATEN ISLAND,WOODROW,96 LENEVAR AVENUE,None,None,False
81401,5,STATEN ISLAND,WOODROW,401 BLOOMINGDALE ROAD,None,None,False
81402,5,STATEN ISLAND,WOODROW,3120 ARTHUR KILL ROAD,None,None,False


In [36]:
# DANGEROUS! HIGH API USAGE!

# OLD
# Apply geolocator function to all rows in updated_df DataFrame
#missing_rows = missing_rows.apply(geolocate, axis=1)


# NEW
#for i in tqdm(missing_rows.index):
    #missing_rows.loc[i] = geolocate(missing_rows.loc[i])
    
# TEST NEW NEW
# # Select the first 10 rows of the DataFrame
# subset = missing_rows.tail(15)

# # Apply the function to the subset
# subset = subset.progress_apply(geolocate, axis=1)

# # Check the results
# print(subset)

# NEW NEW
tqdm.pandas()
missing_rows = missing_rows.progress_apply(geolocate, axis=1)
missing_rows.to_csv('geocodes_export.csv', index=False)

  0%|          | 0/81404 [00:00<?, ?it/s]

In [40]:
# engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}')
upload_geocoding_dataframe_to_sql(engine, 'geocodes', missing_rows)

KeyboardInterrupt: 

In [39]:
# Create the SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}')
# Call the function to print the SQL table
print_sql_table(engine, 'geocodes')

(5, 'STATEN ISLAND', 'WOODROW', '40 HERRICK AVENUE', 40.5307, -74.2192, 0)
(5, 'STATEN ISLAND', 'WOODROW', '104 GLADWIN STREET', 40.5319, -74.2225, 0)
(5, 'STATEN ISLAND', 'WOODROW', '96 LENEVAR AVENUE', 40.5389, -74.2094, 0)
(5, 'STATEN ISLAND', 'WOODROW', '401 BLOOMINGDALE ROAD', 40.5348, -74.218, 0)
(5, 'STATEN ISLAND', 'WOODROW', '3120 ARTHUR KILL ROAD', 40.5438, -74.2335, 0)
(5, 'STATEN ISLAND', 'WOODROW', '123 FAKE STREET', None, None, 1)
(1, 'MANHATTAN', 'ALPHABET CITY', '347 EAST 4TH STREET', 40.7217, -73.9783, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '19 AVENUE D', 40.7207, -73.9785, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '110 AVENUE C', 40.7242, -73.9785, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '326 EAST 4TH STREET', 40.7217, -73.9792, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '328 EAST 4TH STREET', 40.7216, -73.9792, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '719 EAST SIXTH STREET, 1B', 40.7232, -73.9782, 0)
(1, 'MANHATTAN', 'ALPHABET CITY', '271 EAST 7TH STREET', 40.7235, -73.9771, 0)
(1, 'MA

(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 24A', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 34C/E', 40.743, -73.9914, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, PH3C', None, None, 1)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 3D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24 STREET, 18D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 19D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 22D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 27D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 28D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 29D', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 23E', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24TH STREET, 24E', 40.7439, -73.9923, 0)
(1, 'MANHATTAN', 'CHELSEA', '101 WEST 24 STREET, 10H', 40.74

(1, 'MANHATTAN', 'FINANCIAL', '3 HANOVER SQUARE, 8D', 40.705, -74.0097, 0)
(1, 'MANHATTAN', 'FINANCIAL', '55 LIBERTY ST, 23C', 40.7088, -74.0094, 0)
(1, 'MANHATTAN', 'FINANCIAL', '55 LIBERTY STREET, 18CD', None, None, 1)
(1, 'MANHATTAN', 'FINANCIAL', '55 LIBERTY STREET, 21D', 40.7088, -74.0094, 0)
(1, 'MANHATTAN', 'FINANCIAL', '55 LIBERTY STREET, 27BC', None, None, 1)
(1, 'MANHATTAN', 'FINANCIAL', '176 BROADWAY, 7D', None, None, 1)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 602', 40.7052, -74.0085, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 606', 40.7052, -74.0085, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 609', 40.7052, -74.0085, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 613', 40.7075, -74.0117, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 621', 40.7075, -74.0117, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 707', 40.7052, -74.0085, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 709', 40.7052, -74.0085, 0)
(1, 'MANHATTAN', 'FINANCIAL', '1 WALL STREET, 

(1, 'MANHATTAN', 'GRAMERCY', '245 EAST 24TH STREET, 6G', 40.7388, -73.9807, 0)
(1, 'MANHATTAN', 'GRAMERCY', '245 EAST 24TH STREET, 7K/7J', None, None, 1)
(1, 'MANHATTAN', 'GRAMERCY', '242 EAST 25TH STREET, 3B', 40.7392, -73.9806, 0)
(1, 'MANHATTAN', 'GRAMERCY', '242 EAST 25TH STREET, 5E', 40.7392, -73.9806, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH ST, 11DE', 40.7402, -73.9817, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 10G', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 14J', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 15J', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 17H', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 19-A', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 3G', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 3H', 40.7401, -73.9818, 0)
(1, 'MANHATTAN', 'GRAMERCY', '201 EAST 25TH STREET, 

(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 5TH AVENUE, 616', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 5TH AVENUE, 624', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVE, 1623', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVE, 1624', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 1212', 40.7332, -73.9963, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 1220', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 1418', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 15-17', 40.7332, -73.9963, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 304', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 308', 40.7331, -73.996, 0)
(1, 'MANHATTAN', 'GREENWICH VILLAGE-CENTRAL', '24 FIFTH AVENUE, 323', 40.7331, -73.996, 0)


...

(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY WEST, 6F', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY WEST, 9K', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 10C', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 11J', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 3F', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 7M', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 8A', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, 9D', 40.8953, -73.9089, 0)
(2, 'BRONX', 'RIVERDALE', '4705 HENRY HUDSON PARKWAY, STB', None, None, 1)
(2, 'BRONX', 'RIVERDALE', '4901 HENRY HUDSON PARKWAY WEST, 10H', 40.8968, -73.9087, 0)
(2, 'BRONX', 'RIVERDALE', '4901 HENRY HUDSON PARKWAY WEST, 5H', 40.8968, -73.9087, 0)
(2, 'BRONX', 'RIVERDALE', '4901 HENRY HUDSON PARKWAY, 1F', 40.8968, -73.9087, 0)
(2, 'BRONX'

(2, 'BRONX', 'THROGS NECK', '3081 FEARN', 40.8116, -73.8047, 0)
(2, 'BRONX', 'THROGS NECK', '15 SCHUYLER TERRACE', 40.8099, -73.8027, 0)
(2, 'BRONX', 'THROGS NECK', '6 SCHUYLER TERRACE', 40.8097, -73.8027, 0)
(2, 'BRONX', 'THROGS NECK', '1054 REVERE AVENUE', 40.832, -73.8283, 0)
(2, 'BRONX', 'THROGS NECK', '1056 REVERE AVENUE', 40.8321, -73.8283, 0)
(2, 'BRONX', 'THROGS NECK', '922 CALHOUN AVE', 40.8296, -73.8274, 0)
(2, 'BRONX', 'THROGS NECK', '1023 QUINCY AVENUE', 40.8303, -73.8297, 0)
(2, 'BRONX', 'THROGS NECK', '1048 SWINTON AVENUE', 40.8306, -73.8305, 0)
(2, 'BRONX', 'THROGS NECK', '907 QUINCY AVENUE', 40.8287, -73.8283, 0)
(2, 'BRONX', 'THROGS NECK', '1001 SWINTON', 40.8295, -73.83, 0)
(2, 'BRONX', 'THROGS NECK', '2785 BARKLEY', 40.8294, -73.8303, 0)
(2, 'BRONX', 'THROGS NECK', '1036 BRINSMADE AVE', 40.8299, -73.831, 0)
(2, 'BRONX', 'THROGS NECK', '1036 BRINSMADE AVENUE', 40.8299, -73.831, 0)
(2, 'BRONX', 'THROGS NECK', '936 BRINSMADE AVENUE', 40.8287, -73.83, 0)
(2, 'BRONX', 'TH

(3, 'BROOKLYN', 'BATH BEACH', '8649 14TH AVENUE', 40.6117, -74.0126, 0)
(3, 'BROOKLYN', 'BATH BEACH', '77 BAY 7TH STREET', 40.6108, -74.0123, 0)
(3, 'BROOKLYN', 'BATH BEACH', '69 BAY 7TH STREET', 40.6108, -74.0122, 0)
(3, 'BROOKLYN', 'BATH BEACH', '35 BAY 7TH STREET', 40.6115, -74.0116, 0)
(3, 'BROOKLYN', 'BATH BEACH', '38 BAY 8TH STREET', 40.6112, -74.0113, 0)
(3, 'BROOKLYN', 'BATH BEACH', '62 BAY 8TH STREET', 40.6108, -74.0117, 0)
(3, 'BROOKLYN', 'BATH BEACH', '70 BAY 8TH STREET', 40.6106, -74.0118, 0)
(3, 'BROOKLYN', 'BATH BEACH', '63 BAY 8TH STREET', 40.6105, -74.0114, 0)
(3, 'BROOKLYN', 'BATH BEACH', '31 BAY 8TH STREET', 40.6111, -74.0107, 0)
(3, 'BROOKLYN', 'BATH BEACH', '8636 15TH AVENUE', 40.6108, -74.0105, 0)
(3, 'BROOKLYN', 'BATH BEACH', '8642 15TH AVENUE', 40.6107, -74.0106, 0)
(3, 'BROOKLYN', 'BATH BEACH', '1513 BENSON AVENUE', 40.6097, -74.0108, 0)
(3, 'BROOKLYN', 'BATH BEACH', '1509 BENSON AVE', 40.6097, -74.0108, 0)
(3, 'BROOKLYN', 'BATH BEACH', '8657 15TH AVENUE', 40.61

(3, 'BROOKLYN', 'BAY RIDGE', '536 76TH STREET', 40.628, -74.0221, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '7702 6TH AVENUE', 40.6266, -74.0212, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '648 78TH STREET', 40.6249, -74.0201, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '7916 RIDGE BOULEVARD', 40.6289, -74.0321, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '343 85TH STREET', 40.6242, -74.0293, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '242 86TH STREET', 40.6236, -74.0325, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '51 88 STREET', 40.6239, -74.0387, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '345 88TH STREET', 40.6219, -74.0302, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '341 89TH STREET', 40.6213, -74.0305, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '325 89TH STREET', 40.6213, -74.0309, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '253 91ST STREET', 40.6205, -74.0333, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '243 92ND STREET', 40.6198, -74.0339, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '170 92ND STREET', 40.6199, -74.0361, 0)
(3, 'BROOKLYN', 'BAY RIDGE', '132 97TH STREET', 40.6154, -74.0365, 0)
(3, 'BROOKLYN', 'BA

(3, 'BROOKLYN', 'BEDFORD STUYVESANT', '569 JEFFERSON AVENUE, 1', 40.6852, -73.9352, 0)
(3, 'BROOKLYN', 'BEDFORD STUYVESANT', '569 JEFFERSON AVENUE, 2', 40.6852, -73.9352, 0)


In [35]:
missing_rows.to_csv('geocodes_export.csv', index=False)

In [37]:
missing_rows

,BOROUGH CODE,BOROUGH,NEIGHBORHOOD,ADDRESS,LATITUDE,LONGITUDE,GEOCODING ERR
0,1,MANHATTAN,ALPHABET CITY,347 EAST 4TH STREET,40.721665,-73.978312,False
1,1,MANHATTAN,ALPHABET CITY,19 AVENUE D,40.720675,-73.978498,False
2,1,MANHATTAN,ALPHABET CITY,110 AVENUE C,40.724210,-73.978491,False
3,1,MANHATTAN,ALPHABET CITY,326 EAST 4TH STREET,40.721688,-73.979215,False
4,1,MANHATTAN,ALPHABET CITY,328 EAST 4TH STREET,40.721631,-73.979227,False
...,...,...,...,...,...,...,...
81399,5,STATEN ISLAND,WOODROW,104 GLADWIN STREET,40.531902,-74.222495,False
81400,5,STATEN ISLAND,WOODROW,96 LENEVAR AVENUE,40.538871,-74.209380,False
81401,5,STATEN ISLAND,WOODROW,401 BLOOMINGDALE ROAD,40.534780,-74.217978,False
81402,5,STATEN ISLAND,WOODROW,3120 ARTHUR KILL ROAD,40.543765,-74.233477,False


In [66]:
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE geocodes ADD PRIMARY_KEY VARCHAR(255)"))
    connection.execute(text("UPDATE geocodes SET PRIMARY_KEY = CONCAT(`BOROUGH CODE`, '_', `BOROUGH`, '_', `ADDRESS`)"))


In [60]:
text(f'UPDATE {geocodes_sql_table} SET PRIMARY_KEY = CONCAT([BOROUGH CODE], '_', [BOROUGH], '_', [ADDRESS])')






SyntaxError: invalid syntax (<ipython-input-60-90c269e5d287>, line 1)

In [67]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM geocodes LIMIT 2;"))
    tables = [(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7]) for row in result]
    print(tables)

[(5, 'STATEN ISLAND', 'WOODROW', '40 HERRICK AVENUE', 40.5307, -74.2192, 0, '5_STATEN ISLAND_40 HERRICK AVENUE'), (5, 'STATEN ISLAND', 'WOODROW', '104 GLADWIN STREET', 40.5319, -74.2225, 0, '5_STATEN ISLAND_104 GLADWIN STREET')]


In [68]:
with engine.connect() as connection:
    # Add primary key in SQL
    connection.execute(text("ALTER TABLE geocodes ADD PRIMARY_KEY VARCHAR(255)"))

OperationalError: (pymysql.err.OperationalError) (1060, "Duplicate column name 'PRIMARY_KEY'")
[SQL: ALTER TABLE geocodes ADD PRIMARY_KEY VARCHAR(255)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)